In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from subprocess import check_output
from collections import Counter
from sklearn.cross_validation import train_test_split
import xgboost as xgb
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
is_dup = train_data.is_duplicate.value_counts()
is_dup

In [ ]:
stops = set(stopwords.words("english"))

In [ ]:
def word_match_share(row):
    #вычмсл
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R

In [ ]:
train_qs = pd.Series(train_data['question1'].tolist() + train_data['question2'].tolist()).astype(str)
test_qs = pd.Series(test_data['question1'].tolist() + test_data['question2'].tolist()).astype(str)
from collections import Counter
def get_weight(count, eps=10000, min_count=10):
    if count < min_count:
        return 0
    else:
        return 1 / (count + eps)
words = (" ".join(train_qs)).lower().split()
counts = Counter(words)
weights = {word: get_weight(count) for word, count in counts.items()}#сопоставляем слову котрое встретилось больше min_count опред вес

In [ ]:
def tfidf_word_match_share(row):
    #вычисляем отношения веса слов не попавшим в стопслова и находяшихся одновреммено в обоих вопросах к обшему весу

    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
      
        return 0
    
    shared_weights = [weights.get(w, 0) for w in q1words.keys() if w in q2words] + [weights.get(w, 0) for w in q2words.keys() if w in q1words]
    total_weights = [weights.get(w, 0) for w in q1words] + [weights.get(w, 0) for w in q2words]
    
    R = np.sum(shared_weights) / np.sum(total_weights)
    return R

In [ ]:
train_word_match = train_data.apply(word_match_share, axis=1, raw=True)
tfidf_train_word_match = train_data.apply(tfidf_word_match_share, axis=1, raw=True)
x_train = pd.DataFrame()
x_test = pd.DataFrame()
x_train['word_match'] = train_data.apply(word_match_share, axis=1, raw=True)
x_train['tfidf_word_match'] = train_data.apply(tfidf_word_match_share, axis=1, raw=True)
x_test['word_match'] = test_data.apply(word_match_share, axis=1, raw=True)
x_test['tfidf_word_match'] = test_data.apply(tfidf_word_match_share, axis=1, raw=True)

y_train = train_data['is_duplicate'].values #применяем функции

In [ ]:
from sklearn.cross_validation import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=4242)

In [ ]:
import xgboost as xgb


params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 4

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10) #вычисляем бинарную регрессию

In [ ]:
d_test = xgb.DMatrix(x_test)
p_test = bst.predict(d_test)

sub = pd.DataFrame()
sub['test_id'] = test_data['test_id']
sub['is_duplicate'] = p_test
sub.to_csv('simple_xgb.csv', index=False)